In [1]:
import sys
sys.path.append("..")
from data.preparer import load_news_dataset
from babble import Explanation
from babble import BabbleStream
from babble.Candidate import Candidate 

from metal.analysis import lf_summary
from metal.analysis import label_coverage
from metal import LabelModel
from metal.tuners import RandomSearchTuner
from babble.utils import ExplanationIO

import pandas as pd
from datetime import datetime
from snorkel.labeling import filter_unlabeled_dataframe

stat_history = pd.DataFrame()
import nltk
nltk.download("punkt")

import pandas as pd
pd.set_option('display.max_colwidth', -1)
from datetime import datetime
stat_history = pd.DataFrame()

[nltk_data] Downloading package punkt to /Users/sara/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## The Data

These texts discuss either gun politics (1) or computer electronics (0).

If you're not sure about the correct label, that's fine -- either make your best guess or just skip the example.

In [2]:
# Unzip the data. (Don't worry about this, it should be already unzipped.)
# Replace PASSWORD with the password to unzip the data, or download it directly from Kaggle.

#!unzip -P PASSWORD data/data.zip

Load the dataset into training, validation, development, and test sets

In [3]:
df_train, df_dev, df_valid, df_test = load_news_dataset()
print("{} training examples".format(len(df_train)))
print("{} development examples".format(len(df_dev)))
print("{} validation examples".format(len(df_valid)))
print("{} test examples".format(len(df_test)))

477 training examples
400 development examples
100 validation examples
100 test examples


/Users/sara/Documents/GitHub/snorkel-notebooks/venv/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [4]:
dfs = [df_train, df_dev]
dfs[0]['label'] = -1

for df in dfs:
    df["id"] = range(len(df))
    df["label"] += 1

Cs = [df.apply(lambda x: Candidate(x), axis=1) for df in dfs]

# babble labble uses 1 and 2 for labels, while our data uses 0 and 1
# add 1 to convert
Ys = [df.label.values for df in dfs]
Ys[0] -= 1 # no label (training set) should be set to -1

Define the labels for this task.

In [5]:
ABSTAIN = 0
ELECTRONICS = 1
GUNS = 2

# Babble Tutorial
## News forum classification

### You will work with a subset of the 20 NewsGroup dataset. 
The texts shown are from one of two forums:
 1. Computer Electronics (Label 1)
 2. Gun Politics Forum (Label 2)
Your job is to create a training data set to classify texts as belonging to one of these two forums.

You will do this by writing natural language explanations of why you would label an example a certain way (1 (ELECTRONICS), 2 (GUNS), or 0 (ABSTAIN or no label)).
These explanations will be parsed into functions which will be aggregated by Snorkel to create training data from unlabeled examples.

You can evaluate your progress based on the coverage and f1 score of your label model, or by training a logistic regression classifier on the data and evaluating the test result.

In [6]:
# start timer!
stat_history = stat_history.append({"time": datetime.now(), "num_lfs": 0}, ignore_index=True)

,num_lfs,time
0,0.0,2020-02-03 16:11:08.439003


Load the data into a *BabbleStream*: an object that iteratively displays candidates, collects and parses explanations.

In [7]:
# aliases are a way to refer to a set of words in a rule.
aliases = {
    "unit": ["joules", "volts", "ohms", "MHz"]
}


babbler = BabbleStream(Cs, Ys, balanced=True, shuffled=True, seed=456, aliases=aliases)

Grammar construction complete.


In [8]:
def prettyprint(candidate):
    # just a helper function to print the candidate nicely
    print(candidate.text)

Let's look at an example candidate!

In [9]:
# Rerun this cell to get a new example
candidate = babbler.next()
prettyprint(candidate)

This is a followup post to something I've written previously.  Several
people responded with good information, but I don't think I communicated 
exactly what I am looking for.
	I'm working on a custom I/O device that will communicate with a host
via RS-232.  My custom circuitry will use an 80C186EB or EC CPU and require
about 64K of RAM (preferably FLASH RAM).  In looking around, I see that lots
of people have engineered UART-CPU-ROM-RAM subsystems that are ready to be
interfaced to your custom I/O devices.  It's been done so much, that it 
would be best if I can avoid reinventing a system.  It just needs to use
an 80C186 (or 188) CPU, and be able to load a program from the host then 
transfer control to that program.  Well, there's one other thing the ROM
needs to know how to do.  It should have routines to send and receive bytes
to/from the host, that utilize the hardware control lines (DTR,RTS,DTS,CTS).
Everything I've seen is in the $200.00 and up range.  That's too much for
this a

Next, we'll learn how to write a labelling function from a natural language explanation of why you chose a label for a given candidate.

## Create Explanations

Creating explanations generally happens in five steps:
1. View candidates
2. Write explanations
3. Get feedback
4. Update explanations 
5. Apply label aggregator

Steps 3-5 are optional; explanations may be submitted without any feedback on their quality. However, in our experience, observing how well explanations are being parsed and what their accuracy/coverage on a dev set are (if available) can quickly lead to simple improvements that yield significantly more useful labeling functions. Once a few labeling functions have been collected, you can use the label aggregator to identify candidates that are being mislabeled and write additional explanations targeting those failure modes.

Feel free to consult the internet or ask your experiment leader.

*(For the real task, you will be asked to write labeling functions, as quickly and accurately as possible. You will still be allowed to use the internet in this phase, but not ask your experiment leader.)*

### Collection

Use `babbler` to show candidates

In [10]:
candidate = babbler.next()
prettyprint(candidate)
print(candidate.mention_id)


I have the April 15, 1993 issue of the SF Chronicle in my lap.  Page
E7 (in the "Sporting Green" section) has a Trader's advert.  (The
copy is a bit screwed up - it says that the prices offered expire
4-14-93, but the ad is there.)

The SF Examiner and Chronicle run the same set of adverts (because
they have a joint printing/biz agreement and differ only in editorial
content).

I've seen gun ads recently in the merc, which is anti-gun editorially,
albeit not from traders, but from its competitors.

I don't know about the other papers.

Does Traders claim that things are changing?  When?


Because it's easier than telling the truth and no one much cares
either way.


Before you do, make sure that the bozos are actually doing what
you're accusing them of.

-andy
216


Is it about guns or electronics? What makes you think that? (If you don't know, it's okay to make your best guess or skip an example.)

Run the three examples given below, then parse them, and analyze them.
Then, you can try editing them and writing your own functions!

In [11]:
e0 = Explanation(
    # name of this rule, for your reference
    name='electr...', 
    # label to assign
    label=ELECTRONICS, 
    # natural language description of why you label the candidate this way
    condition='A word in the sentence starts with "electr"', 
    # candidate is an optional argument, it should be the id of an example labeled by this rule.
    # if the rule doesn't apply to the candidate you provide, it will be filtered!
    #candidate = candidate.mention_id
)

In [12]:
e1 = Explanation(
    name = 'politics', 
    label = GUNS, 
    condition = 'Any of the words "election", "senator", "democrat", or "republican" are in the text', 
)

In [13]:
e2 = Explanation(
    name = 'selfdefense', 
    label = GUNS, 
    condition = 'because the word "self" occurs before "defense"'
)

Below is an example of an explanation that uses an alias: "unit".

You can define more aliases where the BabbleStream is initialized.

In [14]:
e3 = Explanation(
    name = "units", 
    label = ELECTRONICS, 
    condition = 'A word in the sentence is a unit' 
)

In [15]:
e4 = Explanation(
    name = "e4", 
    label = ABSTAIN, 
    condition = "", 
    candidate = candidate
)

Babble will parse your explanations into functions, then filter out functions that are duplicates, incorrectly label their given candidate, or assign the same label to all examples.

In [16]:
# Add any explanations that you haven't committed yet
explanations = [e0, e1, e2, e3]

parses, filtered = babbler.apply(explanations)

Building list of target candidate ids...
Collected 0 unique target candidate ids from 4 explanations.
No candidate hashes were provided. Skipping linking.
4 explanation(s) out of 4 were parseable.
7 parse(s) generated from 4 explanation(s).
6 parse(s) remain (1 parse(s) removed by DuplicateSemanticsFilter).
Note: 6 LFs did not have candidates and therefore could not be filtered.
6 parse(s) remain (0 parse(s) removed by ConsistencyFilter).
Applying labeling functions to investigate labeling signature.
[========================================] 100%

5 parse(s) remain (1 parse(s) removed by UniformSignatureFilter: (1 None, 0 All)).
3 parse(s) remain (2 parse(s) removed by DuplicateSignatureFilter).
3 parse(s) remain (0 parse(s) removed by LowestCoverageFilter).


### Analysis
See how your parsed explanations performed

In [17]:
try: 
    print(babbler.analyze(parses))
except ValueError as e:
    print("It seems as though none of your labeling functions were parsed. See the cells above and below for more information.")
    print("ERROR:")
    print(e)

             j  Polarity  Coverage  Overlaps  Conflicts  Correct  Incorrect  \
electr..._0  0  1.0       0.0950    0.0050    0.0025     32       6           
politics_0   1  2.0       0.0100    0.0025    0.0025     4        0           
units_0      2  1.0       0.0275    0.0025    0.0000     11       0           

             Emp. Acc.  
electr..._0  0.842105   
politics_0   1.000000   
units_0      1.000000   


See which explanations were filtered and why

In [18]:
babbler.filtered_analysis(filtered)

SUMMARY
4 TOTAL:
0 Unparseable Explanation
1 Duplicate Semantics
0 Inconsistency with Example
1 Uniform Signature
2 Duplicate Signature
0 Lowest Coverage

[#1]: Duplicate Semantics

Parse: return 1 if 'self'.(= 'defense') else 0

Reason: This parse is identical to one produced by the following explanation:
	"because the word "self" occurs before "defense""

Semantics: ('.root', ('.label', ('.int', 2), ('.call', ('.eq', ('.string', 'defense')), ('.string', 'self'))))


[#2]: Uniform Signature

Parse: return 1 if 'self'.(= 'defense') else 0

Reason: This parse labeled NONE of the 400 development examples

Semantics: ('.root', ('.label', ('.int', 2), ('.call', ('.eq', ('.string', 'defense')), ('.string', 'self'))))


[#3]: Duplicate Signature

Parse: return 1 if sum([s.startswith('electr') for s in [w for w in the word(s) the sentence]]).(>= 1) else 0

Reason: This parse labeled identically to the following existing parse:
	Explanation(electr...: 1, "A word in the sentence starts with "el

In [19]:
babbler.commit()

Added 3 parse(s) from 3 explanations to set. (Total # parses = 3)

Applying labeling functions to split 1
[========================================] 100%

Added 53 labels to split 1: L.nnz = 53, L.shape = (400, 3).
Applying labeling functions to split 2
[========================================] 100%

Added 12 labels to split 2: L.nnz = 12, L.shape = (100, 3).


### Evaluation
Get feedback on the performance of your explanations

In [20]:
Ls = [babbler.get_label_matrix(split) for split in [0,1,2]]
lf_names = [lf.__name__ for lf in babbler.get_lfs()]
lf_summary(Ls[1], Ys[1], lf_names=lf_names)

Retrieved label matrix for split 0: L.nnz = 66, L.shape = (477, 3)
Retrieved label matrix for split 1: L.nnz = 53, L.shape = (400, 3)
Retrieved label matrix for split 2: L.nnz = 12, L.shape = (100, 3)


,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
electr..._0,0,1,0.0950,0.0050,0.0025,32,6,0.842105
politics_0,1,2,0.0100,0.0025,0.0025,4,0,1.000000
units_0,2,1,0.0275,0.0025,0.0000,11,0,1.000000


In [21]:
search_space = {
    'n_epochs': [50, 100, 500],
    'lr': {'range': [0.01, 0.001], 'scale': 'log'},
    'show_plots': False,
}

tuner = RandomSearchTuner(LabelModel, seed=123)

label_aggregator = tuner.search(
    search_space, 
    train_args=[Ls[0]], 
    X_dev=Ls[1], Y_dev=Ys[1], 
    max_search=20, verbose=False, metric='f1')

# record statistics over time
pr, re, f1 = label_aggregator.score(Ls[1], Ys[1], metric=['precision', 'recall', 'f1'])
stats = {
    "precision": pr,
    "recall": re,
    "f1": f1,
    "time": datetime.now(),
    "training_label_coverage": label_coverage(Ls[0]),
    "training_label_size": label_coverage(Ls[0])*len(dfs[0])
}
stat_history = stat_history.append(stats, ignore_index=True)

[SUMMARY]
Best model: [2]
Best config: {'n_epochs': 100, 'show_plots': False, 'lr': 0.0011281415605346923, 'seed': 124}
Best score: 0.693069306930693
Precision: 0.530
Recall: 1.000
F1: 0.693


In [22]:
# view some incorrectly labeled examples for a given LF
j = 0
print(lf_names[j])
# set j to match the value of the LF you're interested in
L_dev = Ls[1].todense()
display(df_dev[L_dev[:,j].A1==abs(df_dev["label"]-3)])

units_0


text  \
204  I don't know what Traders is claiming, but it appears to me that\nthe Oakland Tribune has censored gun ads in the past.  Likewise\nfor the San Francisco Chronicle, and I have never seen a gun\nad in the San Francisco Examiner.\n\nSpecifically, about a year ago on Thursdays, when Traders placed\nits ads, the Chron. ad would not have any graphics representing\nany handgun sale, though text could list it.  The Trib. would\nrun a graphic of a handgun.  The Examiner would not have a Traders\nad at all.\n\nOver the past year while Oakland politicians have made a lot of noise\nabout measures to fight crime the Trib stopped taking the Traders\nad, then started publishing it, but without any handgun graphic, then\nstopped, then started.  Since the Trib. was sold some months ago it\nhas not had the Traders ad.  During one of these non-ad interludes\na Traders employee told me that the Trib. had refused to take their\nads.\n\nYes, the usual Chron. Thursday ad was there today, with graphics\nrepresenting rifles, safes, etc. as usual.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

## Train Model
We can train a simple bag of words model on these labels, and see test accuracy.

(This step may take a while).

In [23]:
L_train = Ls[0].todense()
probs_train = label_aggregator.predict_proba(L=L_train)
mask = (L_train != 0).any(axis=1).A1
df_train_filtered = df_train.iloc[mask]
probs_train_filtered = probs_train[mask]
print("{} out of {} examples used for training data".format(len(df_train_filtered), len(df_train)))

62 out of 477 examples used for training data


In [24]:
from analyzer import train_model_from_probs

train_model_from_probs(df_train_filtered, probs_train_filtered, df_valid, df_test)

Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Test Accuracy: 80.0%


## Save
When your time is up, please save your explanations and model!

In [25]:
from types import SimpleNamespace

stat_history.to_csv("babbler_tutorial_statistics_history.csv")
FILE = "babbler_tutorial_explanations.tsv"
exp_io = ExplanationIO()
for exp in explanations:
    if exp.candidate is None:
        exp.candidate = SimpleNamespace(mention_id = None)
exp_io.write(explanations, FILE)
explanations = exp_io.read(FILE)
label_aggregator.save("babble_tutorial_lfmodel.pkl")

Wrote 4 explanations to babbler_tutorial_explanations.tsv
Read 4 explanations from babbler_tutorial_explanations.tsv
